In [1]:
import os
import pandas as pd
import xgboost
import utils
import scoring
from sklearn.model_selection import train_test_split

In [2]:
DATA_PATH = "./data"

In [3]:
train, test = utils.load_data_csv(
    DATA_PATH, utils.SIMPLE_FEATURE_COLUMNS)

/mnt/nkazeev/miniconda3/envs/pip-gpu/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
train.head()

,ncl[0],ncl[1],ncl[2],ncl[3],avg_cs[0],avg_cs[1],avg_cs[2],avg_cs[3],ndof,MatchedHit_TYPE[0],...,Mextra_DX2[3],Mextra_DY2[0],Mextra_DY2[1],Mextra_DY2[2],Mextra_DY2[3],FOI_hits_N,PT,P,label,weight
0,47,31,13,15,2.000000,1.580645,1.153846,1.133333,8,2,...,4295.08540,333.436300,951.68540,2125.09990,4145.18160,6,1764.925899,12646.817591,1,2.226325
1,92,19,11,26,2.750000,2.789474,1.363636,1.230769,8,2,...,777.42914,64.267190,185.53355,413.95690,793.56604,4,4511.281266,28556.630027,1,2.127109
2,100,21,11,12,2.930000,2.428571,1.181818,1.083333,8,2,...,1931.48400,159.182330,440.46994,979.69520,1899.83920,9,1062.550086,17491.702634,1,4.305235
3,74,27,13,20,3.067568,2.518518,1.923077,1.300000,8,2,...,443.21832,36.024845,103.41968,229.75415,438.86194,5,4991.074825,37192.868559,1,1.605583
4,18,22,8,8,1.833333,1.954546,1.125000,1.750000,8,2,...,2430.53470,190.557480,551.34094,1240.64840,2402.91920,6,1347.704888,16688.746174,1,2.055900


In [5]:
test.head()

,ncl[0],ncl[1],ncl[2],ncl[3],avg_cs[0],avg_cs[1],avg_cs[2],avg_cs[3],ndof,MatchedHit_TYPE[0],...,Mextra_DX2[1],Mextra_DX2[2],Mextra_DX2[3],Mextra_DY2[0],Mextra_DY2[1],Mextra_DY2[2],Mextra_DY2[3],FOI_hits_N,PT,P
id,,,,,,,,,,,,,,,,,,,,,
0,55,12,10,15,2.127273,1.166667,1.500000,1.600000,8,2,...,3431.287800,7550.342300,14739.10600,1210.756600,3462.087200,7618.11100,14871.39550,6,1174.484530,6884.502272
1,43,18,12,15,2.697674,1.111111,1.333333,1.600000,8,2,...,63.777058,140.748630,266.41522,21.983400,63.552402,140.25285,265.47680,7,4344.639775,48769.526891
2,90,34,19,29,1.800000,2.058824,1.210526,1.344828,8,2,...,401.787050,903.642400,1749.45590,133.357620,383.406950,862.30440,1669.42520,4,4109.805750,20296.048412
3,141,43,15,16,4.609929,2.744186,1.466667,1.125000,8,2,...,211.605320,471.151430,900.27890,72.982890,211.607040,471.15527,900.28613,9,940.242439,26062.046122
4,22,18,8,9,1.636364,2.444444,1.000000,1.555556,8,2,...,39.059680,85.982086,163.04549,13.728162,39.018433,85.89129,162.87330,8,1386.377851,59872.343332


In [6]:
train_part, validation = train_test_split(train, test_size=0.25, shuffle=True)

In [7]:
model = xgboost.XGBClassifier(n_jobs=-1)

In [ ]:
model.fit(train_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values,
          train_part.label.values,
          sample_weight=train_part.weight.values)

In [ ]:
validation_predictions = model.predict_proba(validation.loc[:, SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [ ]:
scoring.rejection90(validation.label.values, validation_predictions, sample_weight=validation.weight.values)

In [ ]:
model.fit(train.loc[:, SIMPLE_FEATURE_COLUMNS].values, train.label, sample_weight=train.weight)

In [ ]:
predictions = model.predict_proba(test.loc[:, SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [ ]:
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "sample_submission.csv", index_label="id")